In [ ]:
import sys
sys.path.insert(1, '/api/')

In [ ]:
from models.base import Base
from sqlalchemy.orm import backref, column_property, relationship, synonym, validates, sessionmaker
from sqlalchemy import Boolean
from sqlalchemy_utils.functions import drop_database, create_database

In [ ]:
from pydantic import EmailStr, EmailError

In [ ]:
EmailStr().validate("ddd")

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, ForeignKey
url = 'postgresql://admin:password@db:5432/test'
drop_database(url)



In [ ]:
create_database(url)
engine = create_engine(url, echo=True)

In [ ]:
from sqlalchemy import Column, Integer, String

In [ ]:
# with cascade delete on identifier
#deleting the identifier does nothing with cascae delete on Identifier user relation
#deleting the user does not cascade to its related identifiers

In [ ]:
# with cascade delete on user - deleting identifier does nothing
# deleting user deletes the identifiers

In [ ]:
class Subscription(Base):
    __tablename__ = 'subscriptions'

    uuid = Column(UUID(as_uuid=True), primary_key=True,
                  default=uuid4, unique=True, nullable=False)
    identifier_uuid = Column(UUID(as_uuid=True), ForeignKey('personal_identifiers.uuid'),
                             nullable=True)
    identifier = relationship('PersonalIdentifier', foreign_keys=[identifier_uuid])
    #account_uuid = relationship('PersonalIdentifier.account_uuid', foreign_keys=[PersonalIdentifier.account_uuid], cascade='delete')

    # Type of thing being subscribed to
    entity_type = Column(Enum(SubscriptionEntity), nullable=False)
    entity_uuid = Column(UUID(as_uuid=True), nullable=True)


In [ ]:
class Identifier(Base):
  __tablename__ = 'ids'

  id = Column(Integer, primary_key=True)
  value = Column(String)
  verified = Column(Boolean, default=False, nullable=False)
  user_uid = Column(Integer, ForeignKey('users.id'))
  user = relationship('User', foreign_keys=[user_uid], backref='id2')

  def verify(self):
    self.verified = True

In [ ]:
class User(Base):
  __tablename__ = 'users'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  identifiers = relationship('Identifier', foreign_keys=[Identifier.user_uid], cascade='save-update, merge, delete')

  def verify(self):
    for i in self.identifiers:
      i.verified = True

In [ ]:
Session = sessionmaker(binds={
  Identifier:engine,
  User:engine,})
session = Session()

In [ ]:
Base.metadata.create_all(bind=engine)



In [ ]:
u = User(name="tuck")

In [ ]:
i = Identifier(value="tucker@gmailc.om")

In [ ]:
i.user

In [ ]:
session.add(u)
session.commit()

In [ ]:
session.add(i)
session.commit()

In [ ]:
i.verified

In [ ]:
i.verify()

In [ ]:
i.verified

In [ ]:
session.commit()

In [ ]:
session.query(Identifier).first().verified

In [ ]:
i.user_uid = u.id

In [ ]:
i2 = Identifier(value="tucker2@gmail.com")

In [ ]:
i.user

In [ ]:
i2.user_uid = u.id

In [ ]:
session.add(i)

In [ ]:
session.add(i2)

In [ ]:
session.commit()

In [ ]:
u.identifiers[0].verif

In [ ]:
i.user

In [ ]:
i2.user

In [ ]:
u.identifiers

In [ ]:
u.id2

In [ ]:
i.user_uid

In [ ]:
i.user

In [ ]:
session.flush()
session.commit()
session.flush()

In [ ]:
session.query(User).delete()

In [ ]:
session.delete(i)

In [ ]:

session.delete(u)




In [ ]:
session.commit()

In [ ]:
r = session.query(Identifier).all()

In [ ]:
r

In [ ]:
session.query(User).all()

In [ ]:
session.flush()

In [ ]:
session.rollback()